In [2]:
import pandas as pd
akshit_df = './mlchallenge_set_2021.tsv'
akshit_valid = './mlchallenge_set_validation.tsv'
sam_df = './drive/MyDrive/mlchallenge_set_2021_edited.txt'
#sam_valid = './mlchallenge_set_validation.tsv'
#SA_valid=pd.read_table('/Users/shivankagrawal/Documents/ebay/mlchallenge_set_validation.tsv',header=None)
#SA_df=pd.read_table('/Users/shivankagrawal/Documents/ebay/mlchallenge_set_2021.tsv',header=None)
#df=SA_df
#valid=SA_valid
df = pd.read_table(sam_df)
#valid = pd.read_table(sam_valid,sep='\t')
df.columns=['category','primary_image_url','All Links','Tags','index']
#valid.columns=['ID', 'Group']

In [3]:
import re
from collections import Counter
freq=Counter()
attribute=[['']]*len(df)
trialrange=10000
for x in range(trialrange):#range(int(len(df)/10)):#len(df)
    attribute[x]=df.iloc[x,3].lower()
    attribute[x] = re.sub(r'[()]','', attribute[x])
    attribute[x] = re.split(r',', attribute[x])
    attribute[x] = [a.split(':') for a in attribute[x]]
    freq+=Counter([i[0] for i in attribute[x]])
    tempdict={}
    for i in attribute[x]:
            try:
                tempdict[i[0]]=float(i[1])
            except:
                try:
                    tempdict[i[0]]=i[1]
                except:
                    pass
    attribute[x]=tempdict

df['seg']=attribute
#print(df['seg'])

In [ ]:
for i in df['seg'][0:trialrange]:
  if 'color' in i:
    i['color']=re.split(r'[-\s,/]', i['color'])
    #print(i['color'])

In [ ]:
Brands=[]
Images=[]
Colors = []
color_images = []
simple_colors = {'black','white','red','blue','green','yellow','brown','purple','pink','gray','grey'}
print(trialrange)
for i in range(trialrange):
    try:
        for word in df['seg'].iloc[i]['color']:
            if word in simple_colors:
              Colors.append(word)
              color_images.append(df['primary_image_url'].iloc[i])
              break
    except:
        continue
#print(Colors)

In [ ]:
n = 2500
from PIL import Image, ImageFile
import requests
import urllib.request
from io import BytesIO
import numpy as np
import grequests
import pycurl

ImageFile.LOAD_TRUNCATED_IMAGES = True
image_array = []
images = []
max_height = 0
max_width = 0
i = 0

def loadImage(URL):
    with urllib.request.urlopen(URL) as url:
        img = Image.open(BytesIO(url.read()))
    return img

for url in color_images[0:n]:
    if i%500 == 0:
      print(i)
    i+=1
    img = loadImage(url)
    img = img.convert('RGB')
    img = img.resize((200,200))
    pix=np.asarray(img)
    pix=pix.astype('float32')
    pix/=255.0
    images.append(pix)

In [ ]:
color_dict = {}
num = 0
labels = []
for b in Colors:
    if b not in color_dict:
        color_dict[b] = num
        num+=1
    labels.append(color_dict[b])
print(len(labels))
m = 2000
n = round(m*.8)
from collections import Counter
print(Counter(labels[0:2500]))

In [187]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Dropout

model = models.Sequential()
model.add(layers.Conv2D(8, (3, 3), activation='softmax', input_shape=(200, 200, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='softmax'))
model.compile(optimizer=tf.keras.optimizers.Adam(epsilon=.01),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [201]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='softmax', input_shape=(200, 200, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='softmax'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='softmax'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='softmax'))
model.add(layers.Dense(32))

model.compile(optimizer=tf.keras.optimizers.Adam(epsilon=.01),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
m = 2500
history = model.fit(np.asarray(images[0:m]), np.asarray(labels[0:m],np.int16), epochs=50, batch_size = 500,validation_split=.8)

In [ ]:
needs_color = []
for x in df['seg'][0:trialrange]:
  if 'color' not in x.keys() and 'colour' not in x.keys():
      needs_color.append(x)
print(len(needs_color))